In [ ]:
#@title Choose Model for Language Summarization(Audio)
# @markdown BART : For better accuracy, but lenghty (FAST) \
# @markdown T-5  : Summary may be concise, but tends to Hallucinate (SLOW)
# hdjdjd
Model = "BART" # @param ["BART", "T5"]
print(Model)

BART


In [ ]:
from google.colab import drive # loads a library to mount your google drive
drive.mount('/content/drive')
#!rm /content/drive/MyDrive/AVS*
!rm -r /content/drive/MyDrive/AVS/Frames/*
!rm -r /content/drive/MyDrive/AVS/preprocessed_img*
!rm -r /content/drive/MyDrive/AVS/summarized_frames*

!mkdir /content/drive/MyDrive/AVS
!mkdir /content/drive/MyDrive/AVS/preprocessed_img
!mkdir /content/drive/MyDrive/AVS/summarized_frames
!mkdir /content/drive/MyDrive/AVS/Frames
!mkdir /content/drive/MyDrive/AVS/DATA

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
rm: cannot remove '/content/drive/MyDrive/AVS/Frames/*': No such file or directory
mkdir: cannot create directory ‘/content/drive/MyDrive/AVS’: File exists
mkdir: cannot create directory ‘/content/drive/MyDrive/AVS/Frames’: File exists
mkdir: cannot create directory ‘/content/drive/MyDrive/AVS/DATA’: File exists


In [ ]:
#!pip install numpy
#!pip install tensorflow
#!pip install keras
#!pip install scikit-learn
#!pip install matplotlib
#!pip install scipy
#!pip install tkinter
#!pip install tqdm
#!pip install seaborn
#!pip install opencv-python
#!pip install librosa
#!pip install ffmpeg-python
!pip install mutagen
!pip install git+https://github.com/openai/whisper.git  # audio to text
!sudo apt update && sudo apt install ffmpeg
!pip install transformers                               # sumarization
!pip install gTTS

  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-3utnyf2c
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-3utnyf2c
  Resolved https://github.com/openai/whisper.git to commit ba3f3cd54b0e5b8ce1ab3de13e32122d0d5f98ab
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [2,118 kB]
Hit:8 https://ppa.launchpadcontent.net/c2d4u.tea

In [ ]:
import os
import sys
import math
import imageio
import cv2
import random
import pathlib
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from imageio import imread, imsave
from os.path import isfile, join
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.applications import DenseNet121
from tkinter import filedialog
from os import listdir
from mutagen.mp3 import MP3
from PIL import Image
from pathlib import Path
from gtts import gTTS
from IPython.display import Audio
from moviepy import editor
from transformers import pipeline, AutoModelForSeq2SeqLM, AutoTokenizer

In [ ]:
def FRAME_EXT(video_path, output_dir, interval=1):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"Error: Could not find path'")
        return
    os.makedirs(output_dir, exist_ok=True)
    frame_count = 0
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        if frame_count % interval == 0:
            frame_path = os.path.join(output_dir, f"frame_{frame_count}.jpg")
            cv2.imwrite(frame_path, frame)
        frame_count += 1
    cap.release()

In [ ]:
input_directory = "/content/drive/My Drive/AVS/DATA/"
Frame_directory = "/content/drive/My Drive/AVS/Frames/"

# Get the list of files in the input directory and sort them by modification time
videofiles = sorted(os.listdir(input_directory), key=lambda x: os.path.getmtime(os.path.join(input_directory, x)))
i = 0
for filename in videofiles:
    if filename.endswith('.mp4'):
       video_path = os.path.join(input_directory, filename)
       FRAME_EXT(video_path, Frame_directory +str(filename), interval=30)
       file_path = filename
    i+=1
print("Frame extraction is completed")
 #preprocessing using bilateral filter
 #obtaining input image


Frame extraction is completed


In [ ]:
print(file_path,filename,input_directory+file_path)
file_source = input_directory+file_path

48.mp4 48.mp4 /content/drive/My Drive/AVS/DATA/48.mp4


In [ ]:
preprocessimg_path="/content/drive/My Drive/AVS/preprocessed_img/"

i = 0
for filename1 in file_path:
    from pathlib import Path
    gg=Path(file_path).resolve().stem
    video_path1 = os.path.join(Frame_directory, gg+str('.mp4'))
    for filename in  listdir(video_path1):
        video_path = os.path.join(video_path1,str(filename))
        img = cv2.imread(video_path)
        img = cv2.resize(img,[224,224])
        # bilateral filter
        filtered_img1  = cv2.bilateralFilter(img,9,75,75)

        prefn = os.path.join(preprocessimg_path, filename)
        cv2.imwrite(prefn, filtered_img1 )
        mset1 = np.square(np.subtract(img,filtered_img1)).mean()
        MSR1=np.mean(filtered_img1)/np.std(filtered_img1)
        PSNRVAL2=10.0*math.log10((255*255)/mset1)
        print('****** Bilateral filter results ****** ')
        print(MSR1)
        print(PSNRVAL2)
    i+=1
print("Pre processing is completed")

  MSR1=np.mean(filtered_img1)/np.std(filtered_img1)

  PSNRVAL2=10.0*math.log10((255*255)/mset1)



****** Bilateral filter results ****** 
nan
inf
****** Bilateral filter results ****** 
1.8812383749115802
36.864818767827515
****** Bilateral filter results ****** 
1.7998634041893478
36.91314797950663
****** Bilateral filter results ****** 
1.7677051561873438
36.92349654928515
****** Bilateral filter results ****** 
1.837009057237401
36.90294143736751
****** Bilateral filter results ****** 
2.579452488471641
40.675464104944595
****** Bilateral filter results ****** 
1.291823254970366
35.00708674904337
****** Bilateral filter results ****** 
1.2925855927475212
34.91525018410662
****** Bilateral filter results ****** 
1.3390273036264873
34.47538481905449
****** Bilateral filter results ****** 
1.336710163248289
34.48277465459907
****** Bilateral filter results ****** 
1.3352560216693703
34.468487957954025
****** Bilateral filter results ****** 
1.2924734181487945
34.893399665110586
****** Bilateral filter results ****** 
1.2977169350103193
34.90855021119042
****** Bilateral filter resu

In [ ]:
# Feature extraction using DenseNet121 as the base model
def load_data(dir_list, image_size):


    # load all images in a directory
    data = []
    labl = []
    image_width, image_height = image_size

    base_model = DenseNet121(include_top=False,weights=None,classes = 2,input_tensor=None, input_shape=(224,224,3))
    for directory in dir_list:
        for filename in listdir(directory):
            print(filename)
            # load the image
            path=directory  + filename
            print(path)
            image  = cv2.imread(path,3)
            imgk = np.expand_dims(image, axis=0)
            print(np.shape(image))
            resincepfeat_par = base_model.predict(imgk)
            print(np.shape(resincepfeat_par))
            datak= np.array(resincepfeat_par)
            datak= (np.mean(datak,1))
            datak= (np.mean(datak,2))
            datak1= (np.mean(datak))

            if datak1 < 2:
                labl.append([0])
                data.append(datak)

            elif datak1 > 2:
                labl.append([1])
                data.append(datak)

    return data, labl,directory

IMG_WIDTH, IMG_HEIGHT = (224,224)

In [ ]:
preprocessimg_path="/content/drive/My Drive/AVS/preprocessed_img/"
print(listdir(preprocessimg_path))
#model = tf.keras.models.load_model('model.h5')
data, labl,directory = load_data([preprocessimg_path], (IMG_WIDTH, IMG_HEIGHT))
labl = np.array(labl)
data = np.array(data)
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout, GRU, Bidirectional
from keras.optimizers import SGD
import math
from sklearn.metrics import mean_squared_error
AA=np.shape(data)
model = Sequential()
### First GRU layer with Dropout regularisation(Input)
model.add(GRU(units=50, return_sequences=True, input_shape=(1, 7), activation='tanh'))
model.add(Dropout(0.2))
### Second GRU layer(Hidden)
model.add(GRU(units=50, return_sequences=True, input_shape=(1, 7), activation='tanh'))
model.add(Dropout(0.2))
### Third GRU layer(Reset)
model.add(GRU(units=50, return_sequences=True, input_shape=(1, 7), activation='tanh'))
model.add(Dropout(0.2))
### Fourth GRU layer(Update)
model.add(GRU(units=50, activation='tanh'))
model.add(Dropout(0.2))
### The output layer
model.add(Dense(units=1))
import tensorflow as tf
epochs = 50
learning_rate = 0.01
decay_rate = learning_rate / epochs
optimizer = tf.keras.optimizers.legacy.Adam(lr=0.001,decay=decay_rate)
#model.compile(optimizer=SGD(lr=0.001, decay=1e-7, momentum=0.9, nesterov=False),loss='mean_squared_error')
model.compile(optimizer=optimizer,loss='mean_squared_error')
### Fitting to the training set
model.fit(data,labl,epochs=50,batch_size=150)
out = model.predict(data)
### The LSTM architecture
model1 = Sequential()
### First LSTM layer with Dropout regularisation
model1.add(LSTM(units=50, return_sequences=True, input_shape=(1,7)))
model1.add(Dropout(0.2))
### Second LSTM layer
model1.add(LSTM(units=50, return_sequences=True))
model1.add(Dropout(0.2))
### Third LSTM layer
model1.add(LSTM(units=50, return_sequences=True))
model1.add(Dropout(0.2))
### Fourth LSTM layer
model1.add(LSTM(units=50))
model1.add(Dropout(0.2))
### The output layer
model1.add(Dense(units=1))
model1.compile(optimizer='rmsprop',loss='mean_squared_error')
### Fitting to the training set
model1.fit(data,labl,epochs=50,batch_size=32)
out1 = model1.predict(data)

out=np.round(out*1000)
out1=np.round(out1*1000)
aa=np.shape(out)
flab=np.zeros([aa[0],1], dtype=int)
##
for ip in range(0,aa[0]):
    if [out[ip]]==out1[ip]:
        flab[ip]=out[ip]
    else:
        flab[ip]=out1[ip]
##
list=np.unique(flab, return_index=True)
index=list[1]
index=np.sort(index)
print(index)
#print("Accuracy:",metrics.accuracy_score(c1, c))

['frame_0.jpg', 'frame_30.jpg', 'frame_60.jpg', 'frame_90.jpg', 'frame_120.jpg', 'frame_150.jpg', 'frame_180.jpg', 'frame_210.jpg', 'frame_240.jpg', 'frame_270.jpg', 'frame_300.jpg', 'frame_330.jpg', 'frame_360.jpg', 'frame_390.jpg', 'frame_420.jpg', 'frame_450.jpg', 'frame_480.jpg', 'frame_510.jpg', 'frame_540.jpg', 'frame_570.jpg', 'frame_600.jpg', 'frame_630.jpg', 'frame_660.jpg', 'frame_690.jpg', 'frame_720.jpg', 'frame_750.jpg', 'frame_780.jpg', 'frame_810.jpg', 'frame_840.jpg', 'frame_870.jpg', 'frame_900.jpg', 'frame_930.jpg', 'frame_960.jpg', 'frame_990.jpg', 'frame_1020.jpg', 'frame_1050.jpg', 'frame_1080.jpg', 'frame_1110.jpg', 'frame_1140.jpg', 'frame_1170.jpg', 'frame_1200.jpg', 'frame_1230.jpg', 'frame_1260.jpg', 'frame_1290.jpg', 'frame_1320.jpg', 'frame_1350.jpg', 'frame_1380.jpg', 'frame_1410.jpg', 'frame_1440.jpg', 'frame_1470.jpg', 'frame_1500.jpg', 'frame_1530.jpg', 'frame_1560.jpg', 'frame_1590.jpg', 'frame_1620.jpg', 'frame_1650.jpg', 'frame_1680.jpg', 'frame_1710.

  super().__init__(name, **kwargs)



Epoch 1/50
1/1 [==============================] - 9s 9s/step - loss: 0.9904
Epoch 2/50
1/1 [==============================] - 0s 15ms/step - loss: 0.9310
Epoch 3/50
1/1 [==============================] - 0s 14ms/step - loss: 0.8798
Epoch 4/50
1/1 [==============================] - 0s 14ms/step - loss: 0.8154
Epoch 5/50
1/1 [==============================] - 0s 14ms/step - loss: 0.7547
Epoch 6/50
1/1 [==============================] - 0s 13ms/step - loss: 0.7065
Epoch 7/50
1/1 [==============================] - 0s 14ms/step - loss: 0.6393
Epoch 8/50
1/1 [==============================] - 0s 14ms/step - loss: 0.5767
Epoch 9/50
1/1 [==============================] - 0s 16ms/step - loss: 0.5160
Epoch 10/50
1/1 [==============================] - 0s 14ms/step - loss: 0.4270
Epoch 11/50
1/1 [==============================] - 0s 12ms/step - loss: 0.3563
Epoch 12/50
1/1 [==============================] - 0s 14ms/step - loss: 0.2938
Epoch 13/50
1/1 [==============================] - 0s 12ms/step

In [ ]:
print(list)

(array([368, 922, 926, 927, 928, 929, 930, 931, 932, 933, 934, 936, 937,
       938, 940, 941, 945, 951, 957, 961, 962, 963, 964, 965, 973, 974,
       975, 976, 998]), array([ 0, 32, 68, 71, 85, 59,  7,  6, 18, 34, 36, 46, 45, 52, 76, 56, 51,
       92, 83, 22, 19, 95, 21, 24, 96, 93,  2,  1,  5]))


In [ ]:
# summarized frames

preprocessimg_path="/content/drive/My Drive/AVS/preprocessed_img/"
summarizedimg_path="/content/drive/My Drive/AVS/summarized_frames/"
fn=29
for filename in  index:

  #print(fn)
  video_path = preprocessimg_path+'frame_'+str(filename+filename*fn)+'.jpg'
  #fn=fn+29
  print(filename+fn)
  print(video_path)
  img = cv2.imread(video_path)
  prefn = summarizedimg_path+str(filename)+'.jpg'
  cv2.imwrite(prefn, img )
print("Video Summarization is completed")

29
/content/drive/My Drive/AVS/preprocessed_img/frame_0.jpg
30
/content/drive/My Drive/AVS/preprocessed_img/frame_30.jpg
31
/content/drive/My Drive/AVS/preprocessed_img/frame_60.jpg
34
/content/drive/My Drive/AVS/preprocessed_img/frame_150.jpg
35
/content/drive/My Drive/AVS/preprocessed_img/frame_180.jpg
36
/content/drive/My Drive/AVS/preprocessed_img/frame_210.jpg
47
/content/drive/My Drive/AVS/preprocessed_img/frame_540.jpg
48
/content/drive/My Drive/AVS/preprocessed_img/frame_570.jpg
50
/content/drive/My Drive/AVS/preprocessed_img/frame_630.jpg
51
/content/drive/My Drive/AVS/preprocessed_img/frame_660.jpg
53
/content/drive/My Drive/AVS/preprocessed_img/frame_720.jpg
61
/content/drive/My Drive/AVS/preprocessed_img/frame_960.jpg
63
/content/drive/My Drive/AVS/preprocessed_img/frame_1020.jpg
65
/content/drive/My Drive/AVS/preprocessed_img/frame_1080.jpg
74
/content/drive/My Drive/AVS/preprocessed_img/frame_1350.jpg
75
/content/drive/My Drive/AVS/preprocessed_img/frame_1380.jpg
80
/cont

In [ ]:
!pip install git+https://github.com/openai/whisper.git  # audio to text
!sudo apt update && sudo apt install ffmpeg
!pip install transformers                               # sumarization
!pip install gTTS

  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-o89wvyyc
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-o89wvyyc
  Resolved https://github.com/openai/whisper.git to commit ba3f3cd54b0e5b8ce1ab3de13e32122d0d5f98ab
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:6 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy 

In [ ]:
!echo $file_source

/content/drive/My Drive/AVS/DATA/48.mp4


In [ ]:
!cp "$file_source" videoIn.mp4

In [ ]:
!whisper videoIn.mp4 --model medium --language English

100%|█████████████████████████████████████| 1.42G/1.42G [00:15<00:00, 97.8MiB/s]
[00:00.000 --> 00:10.000]  As I'm sure you've learned, your GMC really lives up to its never say never reputation,
[00:10.000 --> 00:12.080]  one that will continue for a long time.
[00:12.080 --> 00:16.080]  Yet there comes a time when the tires your GMC came with need to be replaced.
[00:16.080 --> 00:21.080]  That's why your GMC Certified Service Experts are there to help when it's time for new ones.
[00:21.080 --> 00:25.720]  All tires wear out, but rotation helps them last longer and wear out more evenly so you
[00:25.720 --> 00:28.080]  can replace all four at once.
[00:28.080 --> 00:32.120]  New tires generally grip the road better than tires that have some miles on them.
[00:32.120 --> 00:35.080]  See how this tire is worn more on one side than the other?
[00:35.080 --> 00:38.520]  The owner probably didn't rotate his tires regularly.
[00:38.520 --> 00:44.400]  GMC recommends rotating your tires ev

In [ ]:
input_directory = "/content/videoIn.txt"
#input_directory = "/content/7.txt"
file = open(input_directory,'r')
my_text = file.read()
print(my_text)
file.close()
wordCount = len(my_text.split())
sumMax = wordCount - wordCount//10
sumMin = wordCount//2
print(wordCount,sumMax,sumMin)

As I'm sure you've learned, your GMC really lives up to its never say never reputation,
one that will continue for a long time.
Yet there comes a time when the tires your GMC came with need to be replaced.
That's why your GMC Certified Service Experts are there to help when it's time for new ones.
All tires wear out, but rotation helps them last longer and wear out more evenly so you
can replace all four at once.
New tires generally grip the road better than tires that have some miles on them.
See how this tire is worn more on one side than the other?
The owner probably didn't rotate his tires regularly.
GMC recommends rotating your tires every time you change your oil, or 7,500 miles.
Your Certified Service Experts will use a tread depth measurement tool like this to
see if your tires need replacing.
You can also easily do it yourself using a penny.
Insert the penny into the tire tread.
If the tread reaches Lincoln's forehead, the tread is good.
If not, you'll want to visit the GMC Ce

In [ ]:
if Model == "BART":
  summarizer = pipeline("summarization", model="facebook/bart-large-cnn")
  summary = summarizer(my_text, max_length=sumMax, min_length=sumMin, do_sample=False)
  summaryText = summary[0]['summary_text']
  print("BART : ", summary[0]['summary_text'])
else:
  my_text = "summarise : " + my_text
  model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-large")
  tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-large")
  inputs = tokenizer(my_text, return_tensors="pt")
  outputs = model.generate(**inputs, \
                        min_length=20, \
                        max_new_tokens=512, \
                        length_penalty = 2, \
                        num_beams=16, \
                        temperature=0.9, \
                        no_repeat_ngram_size=2, \
                        #num_return_sequences 2,\
                        early_stopping=True)
  output_text_Flan_t5 = tokenizer.batch_decode(outputs, \
                                               skip_special_tokens=True)
  summaryText = str(output_text_Flan_t5)
  print ("T5 : ",summaryText)

The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(



config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

BART :  GMC recommends rotating your tires every time you change your oil, or 7,500 miles. Your Certified Service Experts will use a tread depth measurement tool to see if your tires need replacing. An unbalanced or improperly balanced tire and wheel can make your car vibrate, as well as create uneven tread wear. Watch out for those sneaky potholes and curbs in your GMC. The best way to keep your tires in good condition is to rotate them regularly and replace them with new ones when they wear out. For more information, visit GMCCertifiedService.com or call 1-800-GMC-Certified (1-844-788-7888) or go to www.GMC.com/Certified. The most recent issue of CNN iReport is on sale now.


In [ ]:
sumCount = len(summaryText.split())
print(sumCount)

112


In [ ]:
file = open("summary.txt",'w')
#file.writelines(summary[0]['summary_text'])
file.writelines(summaryText)
file.close()

In [ ]:
#tts = gTTS(summary[0]['summary_text'])
tts = gTTS(summaryText)
sound_file = "/content/drive/My Drive/out.mp3"
tts.save(sound_file)

Audio(sound_file, autoplay=False)

In [ ]:
summarizedimg_path = "/content/drive/MyDrive/AVS/summarized_frames"

In [ ]:
# fusion


video_path = input_directory
images_path = summarizedimg_path
audio_path = "/content/drive/My Drive/out.mp3"
audio = MP3(audio_path)
audio_length = audio.info.length
list_of_images = []
for image_file in os.listdir(images_path):
    if image_file.endswith('.png') or image_file.endswith('.jpg'):
        image_path = os.path.join(images_path, image_file)
        image = Image.open(image_path).resize((400, 400), Image.ANTIALIAS)
        list_of_images.append(image)
print(audio_length)
duration = audio_length/len(list_of_images)
print(duration)
imageio.mimsave('images.gif', list_of_images, fps=1/duration)

video = editor.VideoFileClip("images.gif")
audio = editor.AudioFileClip(audio_path)
final_video = video.set_audio(audio)
os.chdir("/content/drive/My Drive/AVS/")
final_video.write_videofile(fps=1/duration, codec="libx264", filename="videok.mp4")


  image = Image.open(image_path).resize((400, 400), Image.ANTIALIAS)



61.008
2.1037241379310347
Moviepy - Building video videok.mp4.
MoviePy - Writing audio in videokTEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video videok.mp4



Moviepy - Done !
Moviepy - video ready videok.mp4


In [ ]:
#@title -
from IPython.display import HTML
from base64 import b64encode
mp4 = open('/content/drive/My Drive/AVS/videok.mp4','rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=650 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)

In [ ]:
#!rm -r /content/drive/MyDrive/AVS/preprocessed_img/*
#!rm -r /content/drive/MyDrive/AVS/summarized_frames/*
#!rm -r /content/drive/MyDrive/AVS/Frames/*
#!rm -r /content/drive/MyDrive/AVS/DATA/*

In [ ]:
#!rm /content/drive/MyDrive/AVS/*
#!rm -r /content/drive/MyDrive/AVS/Frames/*